In [5]:
# Import stuff
import torch as t
import numpy as np
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = "notebook_connected"
import plotly.express as px
import einops
import plotly.graph_objects as go 
from functools import partial
import tqdm.auto as tqdm
import circuitsvis as cv
from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, ActivationCache
from transformer_lens.components import Embed, Unembed, LayerNorm, MLP
from fancy_einsum import einsum
from jaxtyping import Float, Int, Bool
import re
import random


In [2]:

def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [3]:
## turn off AD to save memory, since we're focusing on model inference here 
t.set_grad_enabled(False)

device = 'cuda' if t.cuda.is_available() else 'cpu'
model = HookedTransformer.from_pretrained('gpt2-small', device=device)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


Studying the head outputs on different distributions

Create the datasets 

In [87]:
proper_nouns = [
    " Goose", " Church",
    " Google", " Chair",
    " Bag", " Statue",
    " Lamp", " Flower"
]

nouns = [
    " goose", " church",
    " google", " chair",
    " bag", " statue",
    " lamp", " flower"
]

multi_names_religious = [
    " Mary", " Joseph",
    " Abraham", " Paul",
    " Isaac", " Noah",
    " Jacob", " Jesus"
]

multi_names_places = [
    " Paris", " London",
    " Madison", " Phoenix",
    " Devon", " Florence",
    " Austin", " Brooklyn"
]

test = proper_nouns
[model.to_single_token(test[i]) for i in range(len(test))]


[46317, 4564, 3012, 9369, 20127, 43330, 28607, 20025]

In [271]:



def get_dataset(N, names):
    prompts = []
    # List of answers, in the format (correct, incorrect)
    answers = []
    # List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
    answer_tokens = []
    for _ in range(N):
        S1, S2 = random.sample(names,2)
        answers.append((S1,S2))
        answers.append((S2,S1))
        prompt1 = f'When{S1} and{S2} went to the shops,{S2} gave the bag to'
        prompt2 = f'When{S1} and{S2} went to the shops,{S1} gave the bag to'
        prompts.append(prompt1)
        prompts.append(prompt2)

        answer_tok_1 = model.to_single_token(S1)
        answer_tok_2 = model.to_single_token(S2)

        answer_tokens.append((answer_tok_1, answer_tok_2))
        answer_tokens.append((answer_tok_2, answer_tok_1))
    assert len(set([len(model.to_str_tokens(prompt)) for prompt in prompts])) == 1
    answer_tokens = t.tensor(answer_tokens).to(device)
    return prompts, answers, answer_tokens



In [272]:
prompts_rel, ans_rel, ans_toks_rel = get_dataset(10,multi_names_religious)
prompts_pl, ans_pl, ans_toks_pl = get_dataset(10,multi_names_places)
prompts_n, ans_n, ans_toks_n = get_dataset(10,nouns)
prompts_pn, ans_pn, ans_toks_pn = get_dataset(10,proper_nouns)



In [273]:
from rich.table import Table, Column
from rich import print as rprint


In [274]:
control_prompts = ['When John and Mary went to the shops, John gave the bag to',
 'When John and Mary went to the shops, Mary gave the bag to',
 'When Tom and James went to the park, James gave the ball to',
 'When Tom and James went to the park, Tom gave the ball to',
 'When Dan and Sid went to the shops, Sid gave an apple to',
 'When Dan and Sid went to the shops, Dan gave an apple to',
 'After Martin and Amy went to the park, Amy gave a drink to',
 'After Martin and Amy went to the park, Martin gave a drink to']

control_answers = [(' Mary', ' John'),
 (' John', ' Mary'),
 (' Tom', ' James'),
 (' James', ' Tom'),
 (' Dan', ' Sid'),
 (' Sid', ' Dan'),
 (' Martin', ' Amy'),
 (' Amy', ' Martin')]

control_ans_prompts = t.tensor([[ 5335,  1757],
        [ 1757,  5335],
        [ 4186,  3700],
        [ 3700,  4186],
        [ 6035, 15686],
        [15686,  6035],
        [ 5780, 14235],
        [14235,  5780]])



In [275]:
print(model(prompts_n).shape)

def ave_logit_diff(prompts, answer_tokens, per_prompt = False):
    final_logits = model(prompts)[:,-1,:]
    answer_logits = final_logits.gather(dim = -1, index = answer_tokens)
    #print(final_logits.shape)
    answer_logit_diff = answer_logits[:,0] - answer_logits[:,1]
    if per_prompt:
        return answer_logit_diff 
    else:
        return answer_logit_diff.mean()

ave_logit_diff(prompts_n, ans_toks_n)

torch.Size([20, 15, 50257])
torch.Size([20, 50257])


tensor(0.3317)

In [326]:
### Is the logit diff for these two names the right metric? 

### First, try the logit diff between the IO and the average of the actual topk logits for each example prompt 


def ave_logit_diff_topk(prompts, answer_tokens, k, per_prompt = False):
    final_logits = model(prompts)[:,-1,:]
    answer_logits = final_logits.gather(dim = -1, index = answer_tokens)[:,0]

    sample_logits = model(prompts)[:,-1]
    vals, ids = t.topk(sample_logits,k)
    topk_avg = vals.mean(dim=-1)

    answer_logit_diff = answer_logits - topk_avg
    #print(answer_logits, topk_avg)
    if per_prompt:
        return answer_logit_diff 
    else:
        return answer_logit_diff.mean()

ave_logit_diff_topk(prompts_pn, ans_toks_pn,15)





tensor(1.3778)

In [353]:
def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

noun_test = [[k,ave_logit_diff_topk(prompts_n, ans_toks_n,k)] for k in range(25)]

pnoun_test = [[k,ave_logit_diff_topk(prompts_pn, ans_toks_pn,k)] for k in range(25)]

rel_test = [[k,ave_logit_diff_topk(prompts_rel, ans_toks_rel,k)] for k in range(25)]

pl_test = [[k,ave_logit_diff_topk(prompts_pl, ans_toks_pl,k)] for k in range(25)]


tensor([    nan, -0.9559, -0.6326, -0.3639, -0.1371,  0.0610,  0.2581,  0.4444,
         0.6090,  0.7528])

In [354]:
scatter(x= t.tensor(noun_test)[:,0], y = t.tensor(noun_test)[:,1],xaxis = 'k', yaxis = 'topk metric', title = 'Avg logit diff between IO and avg of topk answer logits: nouns')

scatter(x= t.tensor(pnoun_test)[:,0], y = t.tensor(pnoun_test)[:,1],xaxis = 'k', yaxis = 'topk metric', title = 'Avg logit diff between IO and avg of topk answer logits: pnouns')

scatter(x= t.tensor(rel_test)[:,0], y = t.tensor(rel_test)[:,1],xaxis = 'k', yaxis = 'topk metric', title = 'Avg logit diff between IO and avg of topk answer logits: rel')

scatter(x= t.tensor(pl_test)[:,0], y = t.tensor(pl_test)[:,1],xaxis = 'k', yaxis = 'topk metric', title = 'Avg logit diff between IO and avg of topk answer logits: places')


In [ ]:
### could also take the difference between the IO logits and the average of the logits for the other names in the set, to check for correlations among the words 


def ave_logit_diff_topk(prompts, answer_tokens, k, per_prompt = False):
    final_logits = model(prompts)[:,-1,:]
    answer_logits = final_logits.gather(dim = -1, index = answer_tokens)[:,0]

    sample_logits = model(prompts)[:,-1]
    vals, ids = t.topk(sample_logits,k)
    topk_avg = vals.mean(dim=-1)

    answer_logit_diff = answer_logits - topk_avg
    #print(answer_logits, topk_avg)
    if per_prompt:
        return answer_logit_diff 
    else:
        return answer_logit_diff.mean()

ave_logit_diff_topk(prompts_pn, ans_toks_pn,15)

In [276]:
def make_table(prompts, answers, answer_tokens, title):
    cols = [
        "Prompt", 
        Column("Correct", style="rgb(0,200,0) bold"), 
        Column("Incorrect", style="rgb(255,0,0) bold"), 
        Column("Logit Difference", style="bold")
    ]
    logit_diffs = ave_logit_diff(prompts,answer_tokens, per_prompt = True)
    
    ave_logits = ave_logit_diff(prompts,answer_tokens, per_prompt = False)
    logit_diff_table = Table(*cols, title=title + f": Ave logit diff = {ave_logits.item():.3f}")
    logit_diffs = ave_logit_diff(prompts,answer_tokens, per_prompt = True)
    
    ave_logits = ave_logit_diff(prompts,answer_tokens, per_prompt = False)
                                 
    ave_logit_diff(prompts, answer_tokens)
    for prompt, ans, logit_diff in zip(prompts, answers,logit_diffs):
        logit_diff_table.add_row(prompt, ans[0], ans[1], f"{logit_diff.item():.3f}")
    rprint(logit_diff_table)

In [277]:
make_table(control_prompts, control_answers, control_ans_prompts, "Control Names")

torch.Size([8, 50257])
torch.Size([8, 50257])
torch.Size([8, 50257])
torch.Size([8, 50257])
torch.Size([8, 50257])


                                  Control Names: Ave logit diff = 3.552                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                        ┃ Correct ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When John and Mary went to the shops, John gave the bag to    │  Mary   │  John     │ 3.337            │
│ When John and Mary went to the shops, Mary gave the bag to    │  John   │  Mary     │ 3.202            │
│ When Tom and James went to the park, James gave the ball to   │  Tom    │  James    │ 2.709            │
│ When Tom and James went to the park, Tom gave the ball to     │  James  │  Tom      │ 3.797            │
│ When Dan and Sid went to the shops, Sid gave an apple to      │  Dan    │  Sid      │ 1.720            │
│ When Dan and Sid went to the shops, Dan gave an apple to      │  Sid    │  Dan      │ 5.281            │
│ After Martin and Amy went to the park, Amy gave a drink to    │  Martin │  Amy      │ 2.601            │
│ After Martin and Amy went to the park, Martin gave a drink to │  Amy    │  Martin   │ 5.767            │
└───────────────────────────────────────────────────────────────┴─────────┴───────────┴──────────────────┘

In [278]:
#[(model.to_str_tokens(ans_toks_n[k],ans_n[k]), model.to_str_tokens(ans_toks_n[k]),model.to_str_tokens(ans_n[k]), ans_n[k]) for k in range(len(ans_toks_n))]

test = ans_toks_n[10]
test_ans = ans_n[10]
[model.tokenizer.decode(test[i]) for i in range(len(test))], model.to_str_tokens(test), ans_n[10]


([' statue', ' bag'], [' statue', ' bag'], (' statue', ' bag'))

In [279]:
make_table(prompts_n, ans_n, ans_toks_n, "nouns")
make_table(prompts_pn, ans_pn, ans_toks_pn, "proper nouns")
make_table(prompts_rel, ans_rel, ans_toks_rel, "Religious Names")
make_table(prompts_pl, ans_pl, ans_toks_pl, "Place Names")

torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])


                                        nouns: Ave logit diff = 0.332                                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                           ┃ Correct ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When bag and flower went to the shops, flower gave the bag to    │  bag    │  flower   │ 1.145            │
│ When bag and flower went to the shops, bag gave the bag to       │  flower │  bag      │ 0.558            │
│ When statue and bag went to the shops, bag gave the bag to       │  statue │  bag      │ -1.878           │
│ When statue and bag went to the shops, statue gave the bag to    │  bag    │  statue   │ 3.145            │
│ When google and chair went to the shops, chair gave the bag to   │  google │  chair    │ 0.347            │
│ When google and chair went to the shops, google gave the bag to  │  chair  │  google   │ 0.130            │
│ When google and lamp went to the shops, lamp gave the bag to     │  google │  lamp     │ -1.220           │
│ When google and lamp went to the shops, google gave the bag to   │  lamp   │  google   │ -0.124           │
│ When lamp and google went to the shops, google gave the bag to   │  lamp   │  google   │ -0.847           │
│ When lamp and google went to the shops, lamp gave the bag to     │  google │  lamp     │ 0.923            │
│ When statue and bag went to the shops, bag gave the bag to       │  statue │  bag      │ -1.878           │
│ When statue and bag went to the shops, statue gave the bag to    │  bag    │  statue   │ 3.145            │
│ When bag and statue went to the shops, statue gave the bag to    │  bag    │  statue   │ 0.192            │
│ When bag and statue went to the shops, bag gave the bag to       │  statue │  bag      │ -0.594           │
│ When bag and goose went to the shops, goose gave the bag to      │  bag    │  goose    │ 1.759            │
│ When bag and goose went to the shops, bag gave the bag to        │  goose  │  bag      │ 0.804            │
│ When google and statue went to the shops, statue gave the bag to │  google │  statue   │ -1.470           │
│ When google and statue went to the shops, google gave the bag to │  statue │  google   │ 0.640            │
│ When flower and lamp went to the shops, lamp gave the bag to     │  flower │  lamp     │ 0.394            │
│ When flower and lamp went to the shops, flower gave the bag to   │  lamp   │  flower   │ 1.461            │
└──────────────────────────────────────────────────────────────────┴─────────┴───────────┴──────────────────┘

torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])


                                    proper nouns: Ave logit diff = 2.305                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                           ┃ Correct ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When Statue and Goose went to the shops, Goose gave the bag to   │  Statue │  Goose    │ -0.604           │
│ When Statue and Goose went to the shops, Statue gave the bag to  │  Goose  │  Statue   │ 7.116            │
│ When Bag and Google went to the shops, Google gave the bag to    │  Bag    │  Google   │ -0.502           │
│ When Bag and Google went to the shops, Bag gave the bag to       │  Google │  Bag      │ 3.053            │
│ When Google and Bag went to the shops, Bag gave the bag to       │  Google │  Bag      │ 1.784            │
│ When Google and Bag went to the shops, Google gave the bag to    │  Bag    │  Google   │ 0.465            │
│ When Chair and Church went to the shops, Church gave the bag to  │  Chair  │  Church   │ -1.197           │
│ When Chair and Church went to the shops, Chair gave the bag to   │  Church │  Chair    │ 5.078            │
│ When Church and Statue went to the shops, Statue gave the bag to │  Church │  Statue   │ 3.321            │
│ When Church and Statue went to the shops, Church gave the bag to │  Statue │  Church   │ 0.536            │
│ When Bag and Statue went to the shops, Statue gave the bag to    │  Bag    │  Statue   │ 5.912            │
│ When Bag and Statue went to the shops, Bag gave the bag to       │  Statue │  Bag      │ 3.118            │
│ When Flower and Google went to the shops, Google gave the bag to │  Flower │  Google   │ -1.361           │
│ When Flower and Google went to the shops, Flower gave the bag to │  Google │  Flower   │ 5.280            │
│ When Google and Flower went to the shops, Flower gave the bag to │  Google │  Flower   │ 2.611            │
│ When Google and Flower went to the shops, Google gave the bag to │  Flower │  Google   │ 0.059            │
│ When Chair and Statue went to the shops, Statue gave the bag to  │  Chair  │  Statue   │ 3.197            │
│ When Chair and Statue went to the shops, Chair gave the bag to   │  Statue │  Chair    │ 4.001            │
│ When Statue and Lamp went to the shops, Lamp gave the bag to     │  Statue │  Lamp     │ -1.026           │
│ When Statue and Lamp went to the shops, Statue gave the bag to   │  Lamp   │  Statue   │ 5.262            │
└──────────────────────────────────────────────────────────────────┴─────────┴───────────┴──────────────────┘

torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])


                                    Religious Names: Ave logit diff = 3.766                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                             ┃ Correct  ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When Mary and Abraham went to the shops, Abraham gave the bag to   │  Mary    │  Abraham  │ 4.154            │
│ When Mary and Abraham went to the shops, Mary gave the bag to      │  Abraham │  Mary     │ 1.812            │
│ When Abraham and Mary went to the shops, Mary gave the bag to      │  Abraham │  Mary     │ 3.620            │
│ When Abraham and Mary went to the shops, Abraham gave the bag to   │  Mary    │  Abraham  │ 5.660            │
│ When Isaac and Jesus went to the shops, Jesus gave the bag to      │  Isaac   │  Jesus    │ 2.516            │
│ When Isaac and Jesus went to the shops, Isaac gave the bag to      │  Jesus   │  Isaac    │ 5.267            │
│ When Joseph and Mary went to the shops, Mary gave the bag to       │  Joseph  │  Mary     │ 4.952            │
│ When Joseph and Mary went to the shops, Joseph gave the bag to     │  Mary    │  Joseph   │ 2.067            │
│ When Abraham and Joseph went to the shops, Joseph gave the bag to  │  Abraham │  Joseph   │ 3.097            │
│ When Abraham and Joseph went to the shops, Abraham gave the bag to │  Joseph  │  Abraham  │ 5.581            │
│ When Abraham and Jacob went to the shops, Jacob gave the bag to    │  Abraham │  Jacob    │ 3.529            │
│ When Abraham and Jacob went to the shops, Abraham gave the bag to  │  Jacob   │  Abraham  │ 4.518            │
│ When Abraham and Jesus went to the shops, Jesus gave the bag to    │  Abraham │  Jesus    │ 4.904            │
│ When Abraham and Jesus went to the shops, Abraham gave the bag to  │  Jesus   │  Abraham  │ 5.207            │
│ When Joseph and Jesus went to the shops, Jesus gave the bag to     │  Joseph  │  Jesus    │ 3.808            │
│ When Joseph and Jesus went to the shops, Joseph gave the bag to    │  Jesus   │  Joseph   │ 1.881            │
│ When Paul and Jesus went to the shops, Jesus gave the bag to       │  Paul    │  Jesus    │ 3.315            │
│ When Paul and Jesus went to the shops, Paul gave the bag to        │  Jesus   │  Paul     │ 2.728            │
│ When Abraham and Noah went to the shops, Noah gave the bag to      │  Abraham │  Noah     │ 2.320            │
│ When Abraham and Noah went to the shops, Abraham gave the bag to   │  Noah    │  Abraham  │ 4.382            │
└────────────────────────────────────────────────────────────────────┴──────────┴───────────┴──────────────────┘

torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])
torch.Size([20, 50257])


                                        Place Names: Ave logit diff = 3.243                                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Prompt                                                               ┃ Correct   ┃ Incorrect ┃ Logit Difference ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ When Devon and Phoenix went to the shops, Phoenix gave the bag to    │  Devon    │  Phoenix  │ 2.833            │
│ When Devon and Phoenix went to the shops, Devon gave the bag to      │  Phoenix  │  Devon    │ 7.358            │
│ When Austin and London went to the shops, London gave the bag to     │  Austin   │  London   │ 2.329            │
│ When Austin and London went to the shops, Austin gave the bag to     │  London   │  Austin   │ 3.869            │
│ When Paris and Phoenix went to the shops, Phoenix gave the bag to    │  Paris    │  Phoenix  │ 0.388            │
│ When Paris and Phoenix went to the shops, Paris gave the bag to      │  Phoenix  │  Paris    │ 3.778            │
│ When Paris and Phoenix went to the shops, Phoenix gave the bag to    │  Paris    │  Phoenix  │ 0.388            │
│ When Paris and Phoenix went to the shops, Paris gave the bag to      │  Phoenix  │  Paris    │ 3.778            │
│ When London and Austin went to the shops, Austin gave the bag to     │  London   │  Austin   │ -1.930           │
│ When London and Austin went to the shops, London gave the bag to     │  Austin   │  London   │ 4.627            │
│ When Brooklyn and Phoenix went to the shops, Phoenix gave the bag to │  Brooklyn │  Phoenix  │ 0.111            │
│ When Brooklyn and Phoenix went to the shops, Brooklyn gave the bag   │  Phoenix  │  Brooklyn │ 6.009            │
│ to                                                                   │           │           │                  │
│ When Austin and Phoenix went to the shops, Phoenix gave the bag to   │  Austin   │  Phoenix  │ 3.296            │
│ When Austin and Phoenix went to the shops, Austin gave the bag to    │  Phoenix  │  Austin   │ 4.150            │
│ When Florence and Phoenix went to the shops, Phoenix gave the bag to │  Florence │  Phoenix  │ 2.819            │
│ When Florence and Phoenix went to the shops, Florence gave the bag   │  Phoenix  │  Florence │ 5.793            │
│ to                                                                   │           │           │                  │
│ When Devon and Austin went to the shops, Austin gave the bag to      │  Devon    │  Austin   │ 2.070            │
│ When Devon and Austin went to the shops, Devon gave the bag to       │  Austin   │  Devon    │ 8.993            │
│ When London and Florence went to the shops, Florence gave the bag to │  London   │  Florence │ -0.268           │
│ When London and Florence went to the shops, London gave the bag to   │  Florence │  London   │ 4.467            │
└──────────────────────────────────────────────────────────────────────┴───────────┴───────────┴──────────────────┘

Looking at the top-k logits for each prompt

In [ ]:

prompts = prompts_pl
k = 5

for sample_prompt in prompts:
    sample_logits = model(sample_prompt)
    sample_probs = t.softmax(sample_logits[0, -1], dim = -1)
    vals, ids = t.topk(sample_probs,k)

    print(f"Prompt = {sample_prompt}")
    for i in range(k):
            print(f"Top {i}th logit. prob = {vals[i]:.2%}, token = {model.tokenizer.decode(ids[i])}")


Looking at the attention heads on each distribution

In [290]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

noun_tokens = model.to_tokens(prompts_n).to(device)
noun_logits, noun_cache = model.run_with_cache(prompts_n, remove_batch_dim=True)
print(noun_logits.shape)

cv.attention.attention_patterns(tokens=noun_tokens, attention=noun_cache["pattern", 0, "attn"])

torch.Size([20, 15, 50257])
